# Árboles de decisión para el Titanic

En este notebook vamos a construir árboles de clasificación para predecir la supervivencia de los pasajeros del Titanic. Una vez construidos los árboles compararemos los resultados con los de la regresión logística construida anteriormente. En este notebook usaremos los siguientes módulos:

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

Comenzamos cargando los datos de entrenamiento y validación que genereamos en el notebook de la Regresión Logística:

In [ ]:
X_train = pd.read_csv('./data/xtrain_tit.csv')
X_test = pd.read_csv('./data/xtest_tit.csv')
y_train = pd.read_csv('./data/ytrain_tit.csv')
y_test = pd.read_csv('./data/ytest_tit.csv')

Observamos que los datos se han cargado de manera correcta:

In [ ]:
X_train.head()

In [ ]:
X_train.shape

In [ ]:
X_test.head()

In [ ]:
X_test.shape

## Construcción del modelo

Comenzamos construyendo un árbol básico. Empleamos la librería sklearn que ya habíamos utilizado previamente para las dos regresiones. Comenzamos como siempre instanciando el modelo:

In [ ]:
from sklearn import tree
tree_one = tree.DecisionTreeClassifier()

Una vez instancia el modelo, podemos ajustarlo a nuestros datos de entrenamiento:

In [ ]:
tree_one = tree_one.fit(X_train, y_train)

Observamos algunos parámetros como la profundidad o el número mínimo de muestras para que se continuen generando divisiones:

In [ ]:
tree_one.get_depth()

In [ ]:
tree_one.min_samples_split

Podemos representar el árbol de manera visual:

In [ ]:
tree.plot_tree(tree_one,
               feature_names = X_train.columns, 
               filled = True);

Como podemos apreciar este árbol es terriblemente complejo con miles de ramas perdiendo prácticamente toda su explicabilidad. Evaluemos ahora el modelo, para ello comenzamos generando nuestras predicciones en el conjunto de validación:

In [ ]:
y_pred_one = tree_one.predict(X_test)

Calculamos en primer lugar la tasa de acierto en entrenamiento y validación:

In [ ]:
tree_one.score(X_train, y_train)

In [ ]:
tree_one.score(X_test, y_test)

Vemos que estamos incurriendo en algo de overfitting. Hay prácticamente un 15% de diferencia entre entrenamiento y validación. Este es el primer indicador de que el modelo se puede mejorar.

Como ya vimos previamente este dataset está desbalanceado por lo que es necesario calcular otras métricas para poder evaluar de manera correcta la calidad del modelo. Comencemos construyendo la matriz de confusión:

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(y_test, y_pred_one)
print(confusion_matrix)

A partir de esa matriz podemos calcular las métricas específicas:

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred_one))

Por último construimos la curva ROC y calculamos el área bajo la curva:

In [ ]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
logit_roc_auc = roc_auc_score(y_test, tree_one.predict(X_test))
fpr, tpr, thresholds = roc_curve(y_test, tree_one.predict_proba(X_test)[:,1])
plt.figure()
plt.plot(fpr, tpr, label='Primer árbol (Área bajo la curva = %0.2f)' % logit_roc_auc)
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Ratio de falsos positivos')
plt.ylabel('Ratio de verdaderos positivos')
plt.title('Curva ROC')
plt.legend(loc="lower right")
#plt.savefig('Log_ROC') si descomentas esta línea puedes guardar la gráfica
plt.show()

El árbol visto hasta ahora es mejorable en dos sentidos:

* Es un árbol muy complejo con multitud de nodos lo que dificulta enormemente su explicabilidad.
* Sufre de sobreajuste, hay una gran diferencia en la tasa de acierto en entre entrenamiento y validación.

Podemos construir un nuevo árbol en el que reduzcamos los nodos, lo cual mejora estas dos situaciones, los vuelve más explicables y reduce el sobreajuste. Para ello manipulamos dos parámetros: la máxima profundidad, en la que decidimos cuál será la máxima altura que le permitimos crecer al árbol y el número mínimo de muestras para continuar dividiendo los nodos. Para reducir el overfitting fijamos la máxima profundidad en 4 (frente a la altura 17 que tenía el árbol  anterior) y el número mínimo de muestras a 5 frente a las 2 del árbol inicial.

In [ ]:
max_depth = 3
min_samples_split = 5

Una vez fijados los nuevos valores de los parámetros, instanciamos un nuevo árbol y ajustamos el modelo de nuevo a los datos de entrenamiento:

In [ ]:
reduced_tree = tree.DecisionTreeClassifier(max_depth = max_depth, min_samples_split = min_samples_split, random_state = 1)
reduced_tree = reduced_tree.fit(X_train, y_train)

De nuevo recurrimos al conjunto de validación para evaluar la calidad del modelo, para ello comenzamos generando las predicciones en el conjunto de validación:

In [ ]:
y_pred_reduced = reduced_tree.predict(X_test)

Una vez construidas las predicciones podemos comenzar intentando evaluar el overfitting mediante la tasa de acierto:

In [ ]:
reduced_tree.score(X_train, y_train)

In [ ]:
reduced_tree.score(X_test, y_test)

Con esta decisión hemos conseguido atajar el sobreajuste y observamos de hecho que la métrica para validación es ligeramente superior a la de entrenamiento. Y es un 7% superior a la del árbol anterior. Examinemos de nuevo la matriz de confusión:

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(y_test, y_pred_reduced)
print(confusion_matrix)

Vemos que hemos reducido un falso negativo y 8 falsos positivos mejorando sobre todo el número de verdaderos negativos predicho, es decir, reducimos la gente que se clasificaba como superviviente erróneamente.

Observemos las métricas que se pueden extraer de la matriz de confusión para este nuevo árbol:

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred_reduced))

Vemos que la precisión ha mejorado enormemente y el recall mejora para la clase 1 (y se conserva para la 2). El f1 mejora también notablemente. Construimos finalmente la curva ROC:

In [ ]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
logit_roc_auc = roc_auc_score(y_test, reduced_tree.predict(X_test))
fpr, tpr, thresholds = roc_curve(y_test, reduced_tree.predict_proba(X_test)[:,1])
plt.figure()
plt.plot(fpr, tpr, label='Árbol restringido (Área bajo la curva = %0.2f)' % logit_roc_auc)
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Ratio de falsos positivos')
plt.ylabel('Ratio de verdaderos positivos')
plt.title('Curva ROC')
plt.legend(loc="lower right")
#plt.savefig('Log_ROC') si descomentas esta línea puedes guardar la gráfica
plt.show()

Tal y como era de esperar esta métrica mejora respecto al árbol anterior reflejando la mejora del clasificador que ya intuíamos cuando examinamos las demás métricas.

## Interpretación gráfica y explicabilidad

Por último recordemos que el árbol anterior era muy complejo de graficar y explicar. Veamos este árbol en el que hemos limitado la profundidad:

In [ ]:
import graphviz
# DOT data
dot_data = tree.export_graphviz(reduced_tree, out_file=None, 
                                feature_names=X_train.columns,  
                                filled=True)

# Draw graph
graph = graphviz.Source(dot_data, format="png") 
graph

Con este árbol podemos clasificar a todos los pasajeros mediante solo tres preguntas con una precisión del 83%. Un resultado de muy buena calidad.